In [13]:
import pandas as pd  
import numpy as np

In [ ]:
#explores data_example frames
df_bridges = pd.read_excel("../data_example/raw/BMMS_overview.xlsx")
print(df_bridges.columns)
print(df_bridges.head(3))
print("rows:", len(df_bridges))

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')
  road     km              type  LRPName               name  length condition  \
0   N1  1.800       Box Culvert  LRP001a                  .   11.30         A   
1   N1  4.925       Box Culvert  LRP004b                  .    6.60         A   
2   N1  8.976  PC Girder Bridge  LRP008b  Kanch pur Bridge.  394.23         A   

   structureNr                                           roadName chainage  \
0       117861  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8   
1       117862  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925   
2       119889  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    8.976   

   width  constructionYear  spans   zone circle     division   sub-division  \
0   19.5       

In [15]:
df_bridges.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.2,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.2,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1


In [16]:
df_bridges[["km","lat","lon"]].describe()

,km,lat,lon
count,21406.000000,21313.000000,21313.000000
mean,41.217405,23.828505,90.227328
std,78.810955,2.247895,3.364169
min,0.000000,0.000000,0.000000
25%,6.207250,22.942527,89.377806
50%,15.147000,23.823743,90.289194
75%,35.012250,24.720939,91.288417
max,522.718000,91.544194,93.298416


In [17]:
#data_example exploration on missing values, zero values, out of country values etc
#dit is eig cluster 1: de bridges met sws verkeerde waardes 
# 1) Missing coords (NAN values)
mask_missing = df_bridges["lat"].isna() | df_bridges["lon"].isna() 
#--> oplossing: kijken in de andere data_example sets of het daar wel in staat? je weet wel weg waar op zou staan

# 2) Zero coords (0 or 0.0)
mask_zero = (df_bridges["lat"] == 0) | (df_bridges["lon"] == 0) 


# 3) Invalid range coords
mask_invalid = (df_bridges["lat"] < -90) | (df_bridges["lat"] > 90) | (df_bridges["lon"] < -180) | (df_bridges["lon"] > 180)

print("Missing:", mask_missing.sum())
print("Zero:", mask_zero.sum())
print("Invalid:", mask_invalid.sum())

# (optioneel) laat voorbeelden zien
df_bridges.loc[mask_invalid, ["road", "km", "lat", "lon"]].head(10) #dit zijn maar kleine fouten net boven 91 graden niet te veel aandacht aan? 

Missing: 94
Zero: 17
Invalid: 15


,road,km,lat,lon
2864,R241,29.840,91.543889,24.805111
2865,R241,31.212,91.538722,24.816556
2866,R241,32.713,91.530556,24.828056
2867,R241,33.126,91.527917,24.830944
2868,R241,34.978,91.518194,24.844972
2869,R241,36.164,91.514083,24.855083
16183,R241,26.292,91.544194,24.773694
16185,R241,28.857,91.542083,24.796694
17247,R241,28.211,91.541778,24.790944
17248,R241,32.128,91.534361,24.823833


In [18]:
#data_example exploration of out of bangladesh countries, need to find literature etc on what lon and lat bounds are if in bangladesh
LAT_MIN, LAT_MAX = 20.5, 26.8
LON_MIN, LON_MAX = 88.0, 92.8

out_country = df_bridges[
    (df_bridges["lat"].notna()) &
    (df_bridges["lon"].notna()) &
    (
        (df_bridges["lat"] < LAT_MIN) |
        (df_bridges["lat"] > LAT_MAX) |
        (df_bridges["lon"] < LON_MIN) |
        (df_bridges["lon"] > LON_MAX)
    )
]

print("Outside Bangladesh:", len(out_country)) 


Outside Bangladesh: 51


In [19]:
#cleaned df = clean csv 
df_cleaned_roads = pd.read_csv("../data/raw/clean.csv") 
df_cleaned_roads.head()   
df_bridges.columns

#match them on road and LRP name

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')

In [20]:
df_cleaned_roads.columns

Index(['Unnamed: 0', 'road', 'chainage', 'lrp', 'lat', 'lon', 'gap', 'type',
       'name'],
      dtype='object')

In [ ]:
#function used more often in the notebook to normalise the lrp names 
def norm(s):
    return s.astype(str).str.strip().str.upper()

# Normalised keys
df_roads = df_cleaned_roads.copy()
df_br = df_bridges.copy()

df_roads["road_norm"] = norm(df_roads["road"])
df_roads["lrp_norm"]  = norm(df_roads["lrp"])

df_br["road_norm"] = norm(df_br["road"])
df_br["lrp_norm"]  = norm(df_br["LRPName"])

In [32]:
#df_br.describe has: 21407 rows

In [33]:
#df_roads.describe has: #52210 rows

In [ ]:
#cluster 2: bridges not in road files
#bridges missing in roads; #So bridge says it is at a certain place but is not 
#LRP code op die road 
bridge_keys = df_br[["road_norm","lrp_norm"]]
road_keys   = df_roads[["road_norm","lrp_norm"]]

bridges_missing_in_roads = bridge_keys.merge(
    road_keys, on=["road_norm","lrp_norm"], how="left", indicator=True
).query("_merge == 'left_only'").drop(columns="_merge")

print("Bridges (BMMS) missing in roads:", len(bridges_missing_in_roads))
bridges_missing_in_roads.head(20)

Bridges (BMMS) missing in roads: 6552


,road_norm,lrp_norm
0,N1,LRP001A
1,N1,LRP004B
11,N1,LRP013B
12,N1,LRP014A
15,N1,LRP018C
17,N1,LRP021C
18,N1,LRP021D
19,N1,LRP021C
21,N1,LRP023D
22,N1,LRP023C


In [ ]:
#I matched the data_example frames in the next cells

In [ ]:
#repeat the distance function as we might need it later in the notebook
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000  # Straal aarde in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [ ]:
#Load them
# 1. load the clean roads (output notebook on roads)
df_roads = pd.read_csv("../data/raw/clean.csv")  

# 2. Load raw bridge data_example
df_bridges = pd.read_excel("../data_example/raw/BMMS_overview.xlsx")

# making the columns the right data_example type
df_roads['chainage'] = pd.to_numeric(df_roads['chainage'], errors='coerce')
df_bridges['km'] = pd.to_numeric(df_bridges['km'], errors='coerce')
# Ensure that coordinates are numbers #cooerce ensures that if something is not correct, it is not deleted but  
#nan value becomes: so perhaps check later!! which these were and whether they were typos 
for c in ['lat', 'lon']:
    df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
    df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

print(f"roads loaded {len(df_roads)}")
print(f"bridges loaded: {len(df_bridges)}")

Wegen geladen: 52210
Bruggen geladen: 21407


In [ ]:
df_analysis = df_bridges.copy()

# # Create new columns
df_analysis['ideal_lat'] = np.nan
df_analysis['ideal_lon'] = np.nan
df_analysis['offset_meters'] = np.nan
df_analysis['error_flag'] = 'OK' # Start value

# List of well-known roads
known_roads = df_analysis.loc[df_analysis['road'].isin(unique_roads), 'road'].unique()

print("Bezig met berekenen van offsets (dit duurt even)...")

for road_id in known_roads:
    # 1.  Get the road data_example (Cleaned)
    road_data = df_roads[df_roads['road'] == road_id].sort_values('chainage')
    if road_data.empty: continue
        
    # 2. Take the bridges on this road
    mask = df_analysis['road'] == road_id
    bridge_kms = df_analysis.loc[mask, 'km']
    
    # 3. CALCULATE IDEAL LOCATION (Interpolation on chainage)
    # This is the “truth” according to the chainage
    ideal_lats = np.interp(bridge_kms, road_data['chainage'], road_data['lat'])
    ideal_lons = np.interp(bridge_kms, road_data['chainage'], road_data['lon'])
    
    df_analysis.loc[mask, 'ideal_lat'] = ideal_lats
    df_analysis.loc[mask, 'ideal_lon'] = ideal_lons
    
    # 4.  Check whether the bridge is “beyond” the road (Clamping required?)
    max_len = road_data['chainage'].max()
    clamped_mask = mask & (bridge_kms > max_len)
    df_analysis.loc[clamped_mask, 'is_clamped'] = True

# FLAGS

# Step A: Calculate distance (only if GPS is available)
# Haversine between Current GPS and Ideal Location
dists = haversine_dist(
    df_analysis['lat'], df_analysis['lon'],
    df_analysis['ideal_lat'], df_analysis['ideal_lon']
)
df_analysis['offset_meters'] = dists

# Stap B: Define categories

# 1.Unknown road
df_analysis.loc[~df_analysis['road'].isin(unique_roads), 'error_flag'] = 'Unknown Road'

# 2. GPS Missing of in Afrika/Oceaan (Bounding box check)
bad_gps = (df_analysis['lat'].isna()) | (df_analysis['lat'] < 20) | (df_analysis['lat'] > 27)
df_analysis.loc[bad_gps, 'error_flag'] = 'Bad GPS (Africa/Null)'

# 3. Big Offset (>20000m) - Unknown road, probably wrong road or bad GPS
large_offset = (~bad_gps) & (df_analysis['offset_meters'] > 20000) #which threshold should i use here? 
df_analysis.loc[large_offset, 'error_flag'] = 'Large Offset (>20000m)'

# 4. Small Offset (50-500m) - Inaccuracy
small_offset = (~bad_gps) & (df_analysis['offset_meters'] >= 50) & (df_analysis['offset_meters'] <= 500)
df_analysis.loc[small_offset, 'error_flag'] = 'Small Offset (50-500m)'

# Print resultaten voor je rapport
print("\n=== DATA QUALITY DIAGNOSE ===")
print(df_analysis['error_flag'].value_counts())
print(f"\nAantal bruggen die voorbij einde weg liggen (Clamped): {df_analysis['is_clamped'].sum()}")

Bezig met berekenen van offsets (dit duurt even)...

=== DATA QUALITY DIAGNOSE ===
error_flag
OK                        12221
Small Offset (50-500m)     8739
Large Offset (>20000m)      312
Bad GPS (Africa/Null)       135
Name: count, dtype: int64

Aantal bruggen die voorbij einde weg liggen (Clamped): 411


In [ ]:
#only for the bridgest that do not match on LRP name, I anaLysed what can be wrong, as the other ones we already have a solution for 


# Ensure your input data_example frames are correct
# We make copies so we don't have to modify the original
df_b_analysis = df_bridges.copy()
df_r_analysis = df_roads.copy()

# Standardize the column names and types for the merge
# In clean.csv it is called ‘lrp’, in bridges ‘LRPName’ -> we make everything 'LRPName'
df_r_analysis = df_r_analysis.rename(columns={'lrp': 'LRPName'})

# Zorg dat alles strings zijn (voorkomt merge fouten)
df_b_analysis['road'] = df_b_analysis['road'].astype(str)
df_b_analysis['LRPName'] = df_b_analysis['LRPName'].astype(str)
df_r_analysis['road'] = df_r_analysis['road'].astype(str)
df_r_analysis['LRPName'] = df_r_analysis['LRPName'].astype(str)


#MATCH VS NO-MATCH
# Merged to see what matches on lr name and what not
df_merged_check = pd.merge(
    df_b_analysis,
    df_r_analysis[['road', 'LRPName', 'lat']],
    on=['road', 'LRPName'],
    how='left',
    suffixes=('', '_check')
)

# Filter: De unmatched groep (where lat is empty)
mask_unmatched = df_merged_check['lat_check'].isna()
df_unmatched = df_merged_check[mask_unmatched].copy()

#If no match why? 
# List of all known roads 
known_roads = set(df_r_analysis['road'].unique())

def diagnose_reason(row):
    # 1.Does the road even exist in our clean file?
    if row['road'] not in known_roads:
        return "Road Not In Clean File"
    
    # 2. Is the LRP name empty or nan?
    if row['LRPName'] == 'nan' or row['LRPName'] == '' or pd.isna(row['LRPName']):
        return "LRP Name Missing"
    
    # 3. The road exists, the LRP name is there, but it is not in the road file.
    return "LRP Not In Road File"

df_unmatched['reason'] = df_unmatched.apply(diagnose_reason, axis=1)


#Analysis - how bad are the original coordinates? 
# Boundary values for Banglades +/-
min_lat, max_lat = 20.5, 26.7
min_lon, max_lon = 88.0, 92.9

def check_coords(row):
    lat = row['lat']
    lon = row['lon']
    
    # Check on NaN
    if pd.isna(lat) or pd.isna(lon):
        return "Missing Coordinates (NaN)"
    
    # Check on 0,0
    if lat == 0 or lon == 0:
        return "Zero Coordinates (0,0)"
    
    # Check on Foreighn country
    if not (min_lat <= lat <= max_lat) or not (min_lon <= lon <= max_lon):
        return "Outside Bangladesh (Africa/Ocean)"
    
    # If here it seems to be in BD
    return "Coordinates seem OK (Inside BD)"

df_unmatched['coord_quality'] = df_unmatched.apply(check_coords, axis=1)

# print result of bridges 
print("\n" + "="*50)
print("ANALYSE VAN DE NIET-GEMATCHTE BRUGGEN")
print("="*50)

print(f"Totaal aantal bruggen: {len(df_merged_check)}")
print(f"Aantal unmatched:      {len(df_unmatched)} ({len(df_unmatched)/len(df_merged_check):.1%})")

print("\n1. WAAROM GEEN MATCH?")
print("-" * 30)
print(df_unmatched['reason'].value_counts())
print("\n> UITLEG:")
print("> 'LRP Not In Road File': De weg is bekend, dus deze kunnen we fixen met interpolatie (chainage).")
print("> 'Road Not In Clean File': Deze wegen missen in de road-dataset. Deze bruggen kunnen we NIET fixen.")

print("\n2. HOE ZIET DE DATA VAN DEZE GROEP ERUIT?")
print("-" * 30)
print(df_unmatched['coord_quality'].value_counts())

print("\n3. KRUISTABEL: REDEN vs KWALITEIT")
print("-" * 30)
print(pd.crosstab(df_unmatched['reason'], df_unmatched['coord_quality']))

Bezig met diagnosticeren van unmatched bridges...

ANALYSE VAN DE NIET-GEMATCHTE BRUGGEN
Totaal aantal bruggen: 21464
Aantal unmatched:      6568 (30.6%)

1. WAAROM GEEN MATCH?
------------------------------
reason
LRP Not In Road File      6016
Road Not In Clean File     552
Name: count, dtype: int64

> UITLEG:
> 'LRP Not In Road File': De weg is bekend, dus deze kunnen we fixen met interpolatie (chainage).
> 'Road Not In Clean File': Deze wegen missen in de road-dataset. Deze bruggen kunnen we NIET fixen.

2. HOE ZIET DE DATA VAN DEZE GROEP ERUIT?
------------------------------
coord_quality
Coordinates seem OK (Inside BD)      6440
Missing Coordinates (NaN)              94
Outside Bangladesh (Africa/Ocean)      17
Zero Coordinates (0,0)                 17
Name: count, dtype: int64

3. KRUISTABEL: REDEN vs KWALITEIT
------------------------------
coord_quality           Coordinates seem OK (Inside BD)  \
reason                                                    
LRP Not In Road File 

In [ ]:

#analysis of the bridges with a road name that is missing from the road file  
# 1. Standardise Names 
df_bridges['road_norm'] = df_bridges['road'].astype(str).str.strip().str.upper()
df_bridges['lrp_norm'] = df_bridges['LRPName'].astype(str).str.strip().str.upper()

df_roads['road_norm'] = df_roads['road'].astype(str).str.strip().str.upper()
# Check even hoe de kolom heet in jouw clean file (soms 'lrp', soms 'LRPName')
if 'lrp' in df_roads.columns:
    df_roads['lrp_norm'] = df_roads['lrp'].astype(str).str.strip().str.upper()
else:
    df_roads['lrp_norm'] = df_roads['LRPName'].astype(str).str.strip().str.upper()

# 2. Identify unmatched bridges
merged = df_bridges.merge(
    df_roads[['road_norm', 'lrp_norm']], 
    on=['road_norm', 'lrp_norm'], 
    how='left', 
    indicator=True
)

# Filter: only takes bridges not in roads file
unmatched = merged[merged['_merge'] == 'left_only'].copy()

# 3. filter only on roads missing
known_roads = set(df_roads['road_norm'].unique())

# We pakken nu direct de subset waar de weg NIET in die set zit
missing_roads_df = unmatched[~unmatched['road_norm'].isin(known_roads)].copy()

# 4. Determine road type  (N, R, Z)
# Take first letter of road name
missing_roads_df['road_type'] = missing_roads_df['road_norm'].str[0]

# 5. Print result
print(f"Totaal amount of bridges on missing roads: {len(missing_roads_df)}")
print("-" * 40)
print("WELKE TYPEN WEGEN MISSEN WE? (Verdeling N, R, Z)")
print(missing_roads_df['road_type'].value_counts())

print("-" * 40)

#this shows that the bridges on missing roads are mostly Z ways (so local ways), and some Regional roads but not large important 
#national highways where a lot of traffic is likely to use the bridge 


Totaal amount of bridges on missing roads: 552
----------------------------------------
WELKE TYPEN WEGEN MISSEN WE? (Verdeling N, R, Z)
road_type
Z    402
R    150
Name: count, dtype: int64
----------------------------------------


In [42]:

#This checks the chainage mistakes and further analysis them
# for the roads with wrong chainage, so chainage out of bounds of the road length 
#What the mistake looks like the result shows that it might be filled in in meters instead of KM so /10 error

# 1.calculates the lenghts of the roads
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().reset_index()
road_max_lengths.columns = ['road_norm', 'Real_Road_Length']

# 2. takes original bridge data_example with road lengths
df_proof = df_final.merge(road_max_lengths, on='road_norm', how='left')

# 3. calculates changes (Input Chainage - real lenght)
df_proof['Excess_KM'] = df_proof['km'] - df_proof['Real_Road_Length']

# 4. Filter: only bridges outside of road length
bad_chainage = df_proof[df_proof['Excess_KM'] > 0].copy()


# 6. Sort on the biggest mistakes first
bad_chainage = bad_chainage.sort_values(by='Excess_KM', ascending=False)

# 7. Select columns with names
cols_to_show = [
    'road', 
    'LRPName', 
    'km',                # Wat er in de file stond (Fout)
    'Real_Road_Length',  # Hoe lang de weg echt is
    'Excess_KM',         # Hoeveel hij eroverheen gaat
    
]

print(f"Amount of bridges with chainage outside of road lenght {len(bad_chainage)}")
print("Top 20 Extreme Chainage Errors:")
print(bad_chainage[cols_to_show].head(20).to_string(index=False))

Amount of bridges with chainage outside of road lenght 411
Top 20 Extreme Chainage Errors:
 road LRPName    km  Real_Road_Length  Excess_KM
Z7718 LRP042b 42.35             8.150     34.200
Z7718 LRP042a 42.22             8.150     34.070
Z2813 LRP039a 39.18             5.240     33.940
Z2813 LRP037b 37.81             5.240     32.570
Z2813 LRP037a 37.43             5.240     32.190
Z7718 LRP039d 39.58             8.150     31.430
Z2813 LRP036a 36.40             5.240     31.160
Z7718 LRP039b 39.28             8.150     31.130
Z7718 LRP038a 38.93             8.150     30.780
Z7718 LRP038a 38.93             8.150     30.780
Z2812 LRP035c 35.23             5.104     30.126
Z2812 LRP035a 35.12             5.104     30.016
Z2812 LRP034e 34.98             5.104     29.876
Z2812 LRP034c 34.64             5.104     29.536
Z2812 LRP034d 34.29             5.104     29.186
Z2812 LRP034a 34.01             5.104     28.906
Z2812 LRP033c 33.62             5.104     28.516
Z2812 LRP033b 33.53        

In [38]:

#This checks the chainage mistakes and further analysis them
# for the roads with wrong chainage, so chainage out of bounds of the road length 
#What the mistake looks like the result shows that it might be filled in in meters instead of KM so /10 error

#chainage mistakes top 20

# 1. Determine the maximum length of each road in the clean road file.
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().reset_index()
road_max_lengths.columns = ['road_norm', 'Road_Length_Clean_File']

# 2. Filter the bridges that we have snapped to the end

df_errors = df_final[df_final['fix_method'].str.contains("Snapped to End", na=False)].copy()

# 3. Add the road length to the bridges table
df_errors = df_errors.merge(road_max_lengths, on='road_norm', how='left')

# 4.  Calculate the difference (the error)
df_errors['Error_KM'] = df_errors['km'] - df_errors['Road_Length_Clean_File']

# 5. Make table 
tabel_kolommen = [
    'road', 
    'LRPName', 
    'km', 
    'Road_Length_Clean_File', 
    'Error_KM'
]

# Sort from high to low based on the error
fouten_tabel = df_errors[tabel_kolommen].sort_values(by='Error_KM', ascending=False)

print("TOP 20 biggest deviations in chainage (in KM):")
print(fouten_tabel.head(60).to_string(index=False))


#It looks like the comma was misplaced so we have to keep that in mind

TOP 20 biggest deviations in chainage (in KM):
 road LRPName      km  Road_Length_Clean_File  Error_KM
 R114 LRP051e  50.714                  46.436     4.278
 R203 LRP082b  80.287                  76.197     4.090
Z6856 LRP049a  50.428                  46.540     3.888
 R114 LRP051b  50.261                  46.436     3.825
 R114 LRP051a  50.155                  46.436     3.719
Z6856 LRP048b  49.633                  46.540     3.093
Z6856 LRP048a  49.526                  46.540     2.986
 R114 LRP050a  49.171                  46.436     2.735
Z8402 LRP039a  40.171                  37.691     2.480
Z8402 LRP039a  40.171                  37.691     2.480
Z2801 LRP032c  32.781                  30.398     2.383
Z2801 LRP032c  32.781                  30.398     2.383
Z1065 LRP027b  27.924                  25.670     2.254
Z1065 LRP027b  27.924                  25.670     2.254
Z1430 LRP028a  28.172                  26.039     2.133
Z7503 LRP025a  25.561                  23.485     2.076
Z

In [39]:
#final algorithm that makes clean data_example set:
import pandas as pd
import numpy as np


def normalize_text(df, col_name):
    return df[col_name].astype(str).str.strip().str.upper()
# Normalize the key columns in both files
df_bridges['road_norm'] = normalize_text(df_bridges, 'road')
df_bridges['lrp_norm'] = normalize_text(df_bridges, 'LRPName')

df_roads['road_norm'] = normalize_text(df_roads, 'road')
# In clean.csv heet de kolom soms 'lrp' of 'LRPName'
if 'lrp' in df_roads.columns:
    df_roads['lrp_norm'] = normalize_text(df_roads, 'lrp')
else:
    df_roads['lrp_norm'] = normalize_text(df_roads, 'LRPName')



#step 1: clean roads 

# Sorteer wegen (nodig voor interpolatie)
df_roads = df_roads.sort_values(by=['road_norm', 'chainage'])
df_roads = df_roads.drop_duplicates(subset=['road_norm', 'chainage'], keep='first')

# Filter bruggen op onbekende wegen
known_roads = set(df_roads['road_norm'].unique())
original_count = len(df_bridges)
df_final = df_bridges[df_bridges['road_norm'].isin(known_roads)].copy()

print(f"   -> {original_count - len(df_final)} bruggen verwijderd (weg onbekend).")
print(f"   -> {len(df_final)} bruggen over.")

#step 3: look up tables 
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().to_dict()

road_dict = {}
for road_name, group in df_roads.groupby('road_norm'):
    road_dict[road_name] = group[['chainage', 'lat', 'lon']]

#Step 4: Priority match: match on LRP name
df_final = df_final.merge(
    df_roads[['road_norm', 'lrp_norm', 'lat', 'lon']], 
    on=['road_norm', 'lrp_norm'], 
    how='left', 
    suffixes=('', '_matched')
)

#Step 5: Reperation on chaiange
print("5. Start reparatie (LRP -> Chainage -> Smart Typo -> Snap)...")

def repair_bridge_location_final(row):
    #First try matching on LRP name
    if not pd.isna(row['lat_matched']):
        return row['lat_matched'], row['lon_matched'], "Fixed: Matched on LRP Name"
    
    road_id = row['road_norm']
    km = row['km']
    
    if pd.isna(km) or road_id not in road_dict:
        return row['lat'], row['lon'], "Error: Data missing"

    road_data = road_dict[road_id]
    max_len = road_max_lengths[road_id]

    # 2.If that does not work see if chaiange is correct and base lon and lat on that
    if 0 <= km <= max_len:
        new_lat = np.interp(km, road_data['chainage'], road_data['lat'])
        new_lon = np.interp(km, road_data['chainage'], road_data['lon'])
        return new_lat, new_lon, "Fixed: Interpolated by Chainage"

    # 3. chainage mistakes 
    
    # A. small deviation Snap to End
    if km < (max_len * 1.1) or (km - max_len) < 1.0:
        lat_end = np.interp(max_len, road_data['chainage'], road_data['lat'])
        lon_end = np.interp(max_len, road_data['chainage'], road_data['lon'])
        return lat_end, lon_end, "Fixed: Snapped to End (Small deviation)"

    # B. Big deviation Check Typo (Factor 10)
    km_div_10 = km / 10.0
    if km_div_10 <= max_len:
        lat_10 = np.interp(km_div_10, road_data['chainage'], road_data['lat'])
        lon_10 = np.interp(km_div_10, road_data['chainage'], road_data['lon'])
        return lat_10, lon_10, "Fixed: Typo detected (Chainage / 10)"

    # C. Fallback still to big -> Snap to End
    used_km = max(0, min(km, max_len)) # Clamp
    new_lat = np.interp(used_km, road_data['chainage'], road_data['lat'])
    new_lon = np.interp(used_km, road_data['chainage'], road_data['lon'])
    return new_lat, new_lon, "Fixed: Snapped to End (Road too short)"

results = df_final.apply(repair_bridge_location_final, axis=1, result_type='expand')
df_final[['lat_clean', 'lon_clean', 'fix_method']] = results

#export to excel
print(f"6. Opslaan als Excel (Sheet: {SHEET_NAME})...")

original_columns = [
    'road', 'km', 'type', 'LRPName', 'name', 'length', 'condition', 'structureNr', 
    'roadName', 'chainage', 'width', 'constructionYear', 'spans', 'zone', 
    'circle', 'division', 'sub-division', 'lat', 'lon', 'EstimatedLoc'
]

df_export = df_final.copy()
df_export['lat'] = df_export['lat_clean']
df_export['lon'] = df_export['lon_clean']
df_export['EstimatedLoc'] = df_export['fix_method']

# clean columns
for col in ['lat', 'lon', 'km', 'length']:
    df_export[col] = pd.to_numeric(df_export[col], errors='coerce')

df_export = df_export[original_columns]

# sheet_name="BMMS_overview"
#df_export.to_excel(OUTPUT_FILE, index=False, sheet_name=SHEET_NAME)




   -> 552 bruggen verwijderd (weg onbekend).
   -> 20855 bruggen over.
5. Start reparatie (LRP -> Chainage -> Smart Typo -> Snap)...


NameError: name 'SHEET_NAME' is not defined